In [1]:
import pandas as pd
import sqlite3
from fuzzywuzzy import fuzz
import requests




In [2]:
#EXtrére  la base dedonnée
df = pd.read_csv('./project/0_data_source/point_apports_poubelles.csv')
volume_total_par_type = df.groupby(["commune","type_conteneur"])["volume_m3"].sum().reset_index()

nombre_par_type = df[["commune","type_conteneur"]].value_counts().reset_index()


In [3]:
donne = pd.merge(volume_total_par_type, nombre_par_type ,on=["commune","type_conteneur"])

In [4]:
donne.columns= ['commune','type_poubelle', 'volume_total_par_type', 'nombre_par_type']
donne

,commune,type_poubelle,volume_total_par_type,nombre_par_type
0,Bresson,aérien,83.0,24
1,Brié et Angonnes,aérien,50.5,12
2,Brié et Angonnes,enterré,12.0,6
3,Champ sur Drac,aérien,62.0,15
4,Champ sur Drac,enterré,28.0,21
...,...,...,...,...
69,Veurey,aérien,8.0,2
70,Veurey,enterré,12.0,3
71,Vif,aérien,74.0,21
72,Vizille,aérien,275.0,69


In [5]:
population_par_per= pd.read_csv('./project/0_data_source/population_per_commune.csv')
df = pd.merge(donne,population_par_per,on=["commune"])
df

,commune,type_poubelle,volume_total_par_type,nombre_par_type,population
0,Bresson,aérien,83.0,24,664
1,Brié et Angonnes,aérien,50.5,12,2533
2,Brié et Angonnes,enterré,12.0,6,2533
3,Champ sur Drac,aérien,62.0,15,3255
4,Champ sur Drac,enterré,28.0,21,3255
...,...,...,...,...,...
69,Veurey,aérien,8.0,2,1420
70,Veurey,enterré,12.0,3,1420
71,Vif,aérien,74.0,21,8614
72,Vizille,aérien,275.0,69,7305


In [6]:
df['nombre_total_type'] = df.groupby('commune')['nombre_par_type'].transform('sum')
df

,commune,type_poubelle,volume_total_par_type,nombre_par_type,population,nombre_total_type
0,Bresson,aérien,83.0,24,664,24
1,Brié et Angonnes,aérien,50.5,12,2533,18
2,Brié et Angonnes,enterré,12.0,6,2533,18
3,Champ sur Drac,aérien,62.0,15,3255,36
4,Champ sur Drac,enterré,28.0,21,3255,36
...,...,...,...,...,...,...
69,Veurey,aérien,8.0,2,1420,5
70,Veurey,enterré,12.0,3,1420,5
71,Vif,aérien,74.0,21,8614,21
72,Vizille,aérien,275.0,69,7305,78


In [7]:

# Step 2: Calculer le pourcentage de population par type de poubelle
df['pop_per_poub'] = ((df['population']*df['nombre_par_type'])/ df['nombre_total_type'])
df


,commune,type_poubelle,volume_total_par_type,nombre_par_type,population,nombre_total_type,pop_per_poub
0,Bresson,aérien,83.0,24,664,24,664.000000
1,Brié et Angonnes,aérien,50.5,12,2533,18,1688.666667
2,Brié et Angonnes,enterré,12.0,6,2533,18,844.333333
3,Champ sur Drac,aérien,62.0,15,3255,36,1356.250000
4,Champ sur Drac,enterré,28.0,21,3255,36,1898.750000
...,...,...,...,...,...,...,...
69,Veurey,aérien,8.0,2,1420,5,568.000000
70,Veurey,enterré,12.0,3,1420,5,852.000000
71,Vif,aérien,74.0,21,8614,21,8614.000000
72,Vizille,aérien,275.0,69,7305,78,6462.115385


In [8]:
poubelles = df.drop(['nombre_total_type'], axis=1)
poubelles

,commune,type_poubelle,volume_total_par_type,nombre_par_type,population,pop_per_poub
0,Bresson,aérien,83.0,24,664,664.000000
1,Brié et Angonnes,aérien,50.5,12,2533,1688.666667
2,Brié et Angonnes,enterré,12.0,6,2533,844.333333
3,Champ sur Drac,aérien,62.0,15,3255,1356.250000
4,Champ sur Drac,enterré,28.0,21,3255,1898.750000
...,...,...,...,...,...,...
69,Veurey,aérien,8.0,2,1420,568.000000
70,Veurey,enterré,12.0,3,1420,852.000000
71,Vif,aérien,74.0,21,8614,8614.000000
72,Vizille,aérien,275.0,69,7305,6462.115385


In [9]:
conn = sqlite3.connect('./project/1_destination/poubelle_metro.db')
curs = conn.cursor()
# Step 2: Add new columns to the poubelle table
try:
    curs.execute("ALTER TABLE poubelle ADD COLUMN population INTEGER")
    print("Added 'population' column to 'poubelle' table.")
except sqlite3.OperationalError:
    print("'population' column already exists.")


'population' column already exists.


In [10]:

try:
    curs.execute("ALTER TABLE poubelle ADD COLUMN pop_per_poub REAL")
    print("Added 'pop_per_poub' column to 'poubelle' table.")
except sqlite3.OperationalError:
    print("'pop_per_poub' column already exists.")


'pop_per_poub' column already exists.


In [11]:

conn = sqlite3.connect('./project/1_destination/poubelle_metro.db')
curs = conn.cursor()
# Load the transformed data into the 'poubelle' table
# Use 'append' mode to add data to the existing table
poubelles.to_sql('poubelle', conn, if_exists='append', index=False)

# Commit and close the connection
conn.commit()


print("Data loaded into the 'poubelle' table successfully.")

Data loaded into the 'poubelle' table successfully.


In [12]:
for i, row in df.iterrows():
    # Requête SQL pour mettre à jour les colonnes avec les valeurs du DataFrame
    query = """
    UPDATE poubelle
    SET population = ?
    WHERE commune = ? -- Utilise une colonne pour identifier la ligne à mettre à jour
    """
    # Exécuter la requête en passant les valeurs du DataFrame
    curs.execute(query, (row['population'], row['commune']))


conn.commit()

1.4 — Get GPS coordinates lon and lat per municipality (C)

In [13]:
import sqlite3
import pandas as pd


# Étape 1: Charger le fichier CSV avec les coordonnées GPS des villes
ville = pd.read_csv('./project/0_data_source/villes_france_etl.csv')
ville

,insee_code,city_code,zip_code,label,latitude,longitude,department_name,department_number,region_name,region_geojson_name
0,25620,ville du pont,25650,ville du pont,46.999873,6.498147,doubs,25,bourgogne-franche-comté,Bourgogne-Franche-Comté
1,25624,villers grelot,25640,villers grelot,47.361512,6.235167,doubs,25,bourgogne-franche-comté,Bourgogne-Franche-Comté
2,25615,villars les blamont,25310,villars les blamont,47.368384,6.871415,doubs,25,bourgogne-franche-comté,Bourgogne-Franche-Comté
3,25619,les villedieu,25240,les villedieu,46.713906,6.265831,doubs,25,bourgogne-franche-comté,Bourgogne-Franche-Comté
4,25622,villers buzon,25170,villers buzon,47.228558,5.852187,doubs,25,bourgogne-franche-comté,Bourgogne-Franche-Comté
...,...,...,...,...,...,...,...,...,...,...
39140,98829,thio,98829,thio,NaN,NaN,nouvelle-calédonie,988,nouvelle-calédonie,Nouvelle Calédonie
39141,98831,voh,98833,voh,NaN,NaN,nouvelle-calédonie,988,nouvelle-calédonie,Nouvelle Calédonie
39142,98832,yate,98834,yate,NaN,NaN,nouvelle-calédonie,988,nouvelle-calédonie,Nouvelle Calédonie
39143,98612,sigave,98620,sigave,-14.270411,-178.155263,wallis-et-futuna,986,wallis-et-futuna,Wallis-et-Futuna


In [14]:
ville.columns

Index(['insee_code', 'city_code', 'zip_code', 'label', 'latitude', 'longitude',
       'department_name', 'department_number', 'region_name',
       'region_geojson_name'],
      dtype='object')

In [15]:

# Assurez-vous que les noms des colonnes correspondent (les ajuster si nécessaire)
# Exemple : villes_france_etl.csv
base_extraire = ville.loc[:,["city_code","latitude","longitude"]]
base_extraire    

,city_code,latitude,longitude
0,ville du pont,46.999873,6.498147
1,villers grelot,47.361512,6.235167
2,villars les blamont,47.368384,6.871415
3,les villedieu,46.713906,6.265831
4,villers buzon,47.228558,5.852187
...,...,...,...
39140,thio,NaN,NaN
39141,voh,NaN,NaN
39142,yate,NaN,NaN
39143,sigave,-14.270411,-178.155263


In [16]:

# renommé les colonnes 
base_extraire.columns=('commune','gps_lon', 'gps_lat')
base_extraire


,commune,gps_lon,gps_lat
0,ville du pont,46.999873,6.498147
1,villers grelot,47.361512,6.235167
2,villars les blamont,47.368384,6.871415
3,les villedieu,46.713906,6.265831
4,villers buzon,47.228558,5.852187
...,...,...,...
39140,thio,NaN,NaN
39141,voh,NaN,NaN
39142,yate,NaN,NaN
39143,sigave,-14.270411,-178.155263


In [17]:
patterns = ville["label"]
patterns

0              ville du pont
1             villers grelot
2        villars les blamont
3              les villedieu
4              villers buzon
                ...         
39140                   thio
39141                    voh
39142                   yate
39143                 sigave
39144                   uvea
Name: label, Length: 39145, dtype: object

In [109]:
municipality = poubelles["commune"]


0              Bresson
1     Brié et Angonnes
2     Brié et Angonnes
3       Champ sur Drac
4       Champ sur Drac
            ...       
69              Veurey
70              Veurey
71                 Vif
72             Vizille
73             Vizille
Name: commune, Length: 74, dtype: object

In [18]:
from fuzzywuzzy import process
for commune in municipality:

    # Trouver la meilleure correspondance dans 'patterns'

    best_match = process.extractOne(commune, patterns)

    

    if best_match:

        print(f"Commune: {commune} - Best Match: {best_match[0]} with score {best_match[1]}")

        # Étape suivante : mise à jour de la table 'population_gps' avec 'best_match'

        # Vous pouvez ajouter ici le code de mise à jour de la table si nécessaire

NameError: name 'municipality' is not defined

In [ ]:

# Étape 2: Connexion à la base de données SQLite et lecture des communes de la table 'poubelle'
conn = sqlite3.connect('./project/1_destination/poubelle_metro.db')
cursor = conn.cursor()


In [ ]:

# Lire les communes distinctes de la table 'poubelle'
query = "SELECT DISTINCT commune, population FROM poubelle"
poubelle_df = pd.read_sql_query(query, conn)
poubelle_df.head(5)




,commune,population
0,Bresson,664
1,Brié et Angonnes,2533
2,Champ sur Drac,3255
3,Champagnier,1306
4,Claix,1064


In [98]:
# Étape 3: Faire une jointure entre les communes de 'poubelle' et les villes du fichier CSV
# Attention à bien gérer les différences de casse ou d'accents
merged_df = pd.merge(poubelle_df,base_extraire, how="inner",on=["commune"])
merged_df.head(5)

,commune,population,gps_lon,gps_lat


In [89]:

# Étape 3: Faire une jointure entre les communes de 'poubelle' et les villes du fichier CSV
# Attention à bien gérer les différences de casse ou d'accents
merged_df = pd.merge(poubelle_df,base_extraire, how= "left", on=["commune"])
merged_df.head(5)

,commune,population,gps_lon,gps_lat
0,Bresson,664,NaN,NaN
1,Brié et Angonnes,2533,NaN,NaN
2,Champ sur Drac,3255,NaN,NaN
3,Champagnier,1306,NaN,NaN
4,Claix,1064,NaN,NaN


In [90]:
# Étape 3: Faire une jointure entre les communes de 'poubelle' et les villes du fichier CSV
# Attention à bien gérer les différences de casse ou d'accents
merged_df = pd.merge(poubelle_df,base_extraire, how= "right", on=["commune"])
merged_df

,commune,population,gps_lon,gps_lat
0,ville du pont,NaN,46.999873,6.498147
1,villers grelot,NaN,47.361512,6.235167
2,villars les blamont,NaN,47.368384,6.871415
3,les villedieu,NaN,46.713906,6.265831
4,villers buzon,NaN,47.228558,5.852187
...,...,...,...,...
39140,thio,NaN,NaN,NaN
39141,voh,NaN,NaN,NaN
39142,yate,NaN,NaN,NaN
39143,sigave,NaN,-14.270411,-178.155263


In [97]:

# Étape 4: Filtrer uniquement les colonnes nécessaires (nom de la commune, latitude, longitude)
population_gps= merged_df[['commune', 'gps_lon', 'gps_lat']].drop_duplicates()
population_gps

,commune,gps_lon,gps_lat
0,Bresson,NaN,NaN
1,Brié et Angonnes,NaN,NaN
2,Champ sur Drac,NaN,NaN
3,Champagnier,NaN,NaN
4,Claix,NaN,NaN
...,...,...,...
39188,thio,NaN,NaN
39189,voh,NaN,NaN
39190,yate,NaN,NaN
39191,sigave,-14.270411,-178.155263


In [76]:

# Étape 5: Charger les résultats dans la table 'population_gps'
# Créer la table 'population_gps' si elle n'existe pas déjà
create_table_query = """
CREATE TABLE IF NOT EXISTS population_gps (
    commune TEXT PRIMARY KEY,
    latitude REAL,
    longitude REAL
)
"""
cursor.execute(create_table_query)


In [77]:

# Insérer les données dans la table 'population_gps'
for index, row in merged_df.iterrows():
    insert_query = """
    INSERT OR IGNORE INTO population_gps (commune, latitude, longitude)
    VALUES (?, ?, ?)
    """
    cursor.execute(insert_query, (row['commune'], row['latitude'], row['longitude']))

# Commit les changements et fermer la connexion
conn.commit()
conn.close()

print("Insertion réussie des communes avec leurs coordonnées GPS dans la table 'population_gps'.")


Insertion réussie des communes avec leurs coordonnées GPS dans la table 'population_gps'.
